In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression


import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')
import os
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'

In [2]:
df_mod = pd.read_csv("C://Flavio/Renovação/Modelo/Base_mod_novo_mp_dez23_v2.csv", encoding='latin-1',delimiter='|')

In [3]:
df_mod_2=df_mod[["ind_renovacao","premio_ofertado","classe_bonus"]]

In [4]:
df_mod_2['classe_bonus'] = df_mod_2['classe_bonus'].astype('category')
df_mod_2["premio"] = df_mod_2["premio_ofertado"]

In [5]:
df_train = df_mod_2.sample(frac =.80) 
df_test = df_mod_2.sample(frac =.20) 

In [6]:
import statsmodels.formula.api as smf
#Modelo GLM para retenção
formula = 'ind_renovacao ~ np.log(premio+1) + classe_bonus'
log_reg_1 = smf.logit(formula , data=df_train).fit()

Optimization terminated successfully.
         Current function value: 0.641902
         Iterations 5


In [10]:
log_reg_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               359491
Model:                          Logit   Df Residuals:                   359479
Method:                           MLE   Df Model:                           11
Date:                Fri, 26 Apr 2024   Pseudo R-squ.:                 0.06534
Time:                        12:29:05   Log-Likelihood:            -2.3075e+05
converged:                       True   LL-Null:                   -2.4688e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              6.6948      0.069     97.240      0.000       6.560       6.830
classe_bonus[T.1]     -0.0388      0.034     -1.129      0.259      -0.106       0.029
classe_bonus[T.2]      0.4404      0.034     12.837      0.000       0.373       0.508
classe_bonus[T.3]      0.6192      0.035     17.732      0.000       0.551       0.688
classe_bonus[T.4]      0.6907      0.035     19.500      0.000       0.621       0.760
classe_bonus[T.5]      0.6478      0.036     18.146      0.000       0.578       0.718
classe_bonus[T.6]      0.6674      0.036     18.535      0.000       0.597       0.738
classe_bonus[T.7]      0.6358      0.036     17.581      0.000       0.565       0.707
classe_bonus[T.8]      0.6023      0.036     16.553      0.000       0.531       0.674
classe_bonus[T.9]      0.6407      0.036     17.914      0.000       0.571       0.711
classe_bonus[T.10]     0.6925      0.034     20.557      0.000       0.626       0.759
np.log(premio + 1)    -0.9538      0.007   -130.320      0.000      -0.968      -0.939
======================================================================================
"""

In [7]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Preditos
pred_1 = log_reg_1.predict(df_test)
# round() para arredondar para o inteiro proximo;
# 0.5 arredonda para 0; 0.501 arredonda para 1
prediction_1 = list(map(round, pred_1))

# Accuracy score
print('\nTest accuracy = ', accuracy_score(df_test["ind_renovacao"], prediction_1))


Test accuracy =  0.6337387201940516


### Base Para escorar e otimizar

In [8]:
df_lote = pd.read_excel("C://Flavio/Renovação/Aderencia/base_lote202402_v2.xlsx",sheet_name='Base Otim enviada')

In [15]:
df_lote_teste=df_lote[['Act_cost',"Premio_Ofertado_Atuarial","classe_bonus","Pr_anterior_TI"]]
df_lote_teste["premio"]=df_lote_teste["Premio_Ofertado_Atuarial"]
df_lote_teste["classe_bonus"]=df_lote_teste['classe_bonus'].astype('category')
df_lote_teste = df_lote_teste.sample(n = 10) 

In [16]:
df_lote_teste.shape

(10, 5)

In [17]:
df_lote_teste["prob_est"]=log_reg_1.predict(df_lote_teste)

In [18]:
from scipy.optimize import minimize

In [19]:
df_lote_otim=df_lote_teste.copy()

In [20]:
df_lote_otim.columns

Index(['Act_cost', 'Premio_Ofertado_Atuarial', 'classe_bonus',
       'Pr_anterior_TI', 'premio', 'prob_est'],
      dtype='object')

In [30]:
renewal_before = np.mean(df_lote_otim["prob_est"])
renewal_before

0.47923948843682096

In [41]:
# Calculate renewal before optimization
#revenue_before = np.mean(df_lote_teste["premio"])

# Define the objective function (maximize revenue)
#def objective_function(prices,renewal,beta):
def objective_function(prices,renewal):
    df_lote_otim["premio"]=prices
    renewal= log_reg_1.predict(df_lote_otim)
    cost= df_lote_otim["Act_cost"]
    margem_esperada = renewal * (prices-cost)
    #penality = df_lote_otim.shape[0]*((np.mean(prob_esp) - renewal)**2)
    margin = -np.sum(margem_esperada)
    return margin

# Define constraints
def constraint(prices, renewal, min_renewal):
    df_lote_otim["premio"]=prices
    renewal = log_reg_1.predict(df_lote_otim)
    return np.concatenate([[np.mean(renewal) - min_renewal]])
    

# Define initial prices (use predicted prices as initial guess)
initial_prices = df_lote_otim["premio"]

# Minimum revenue constraint
#min_renewal = np.mean(df_lote_otim["prob_est"]) * 1.05
min_renewal = 0.5


# Adjust constraints to ensure prices are no lower than predicted and no greater than 1.2 times predicted
bounds = [(0.7 * base, 2.3 * base) for base in df_lote_otim["Pr_anterior_TI"]]

#renewal_target=0.50

# Perform optimization
result = minimize(objective_function, 
                  initial_prices, 
                  constraints={'type': 'ineq', 'fun': constraint, 'args': (renewal,min_renewal)}, 
                  method = "SLSQP",
                  tol=0.1,
                  bounds=bounds)



# Get optimized prices
optimized_prices = result.x
df_lote_otim["premio"]=result.x
    
#renewal = log_reg_1.predict(df_lote_otim)*np.exp(-beta*np.log(optimized_prices))
    
renewal = log_reg_1.predict(df_lote_otim)

TypeError: objective_function() missing 1 required positional argument: 'renewal'

In [31]:
np.mean(df_lote_otim["prob_est"]) * 1.05

0.5032014628586621

In [299]:
margrem_ini = df_lote_otim["prob_est"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])
np.sum(margrem_ini)

9315651.228465548

In [300]:
margrem_otim = renewal*(optimized_prices-df_lote_otim["Act_cost"])
np.sum(margrem_otim)

9314385.73784348

In [301]:
constraint(optimized_prices,  [renewal_target])

array([-0.02140875])

In [302]:
# Calculate revenue after optimization
revenue_after = np.mean(optimized_prices)

# Print revenue before and after optimization
print("Revenue before optimization:", revenue_before)
print("Revenue after optimization:", revenue_after)

Revenue before optimization: 3022.576157420089
Revenue after optimization: 3023.322401952927


In [303]:
np.mean(renewal)

0.4285912460901649

In [ ]:
# Define revenue before and after optimization
revenue_values = [revenue_before, revenue_after]

# Define labels for the bars
labels = ['Revenue Before Optimization', 'Revenue After Optimization']

# Plotting
plt.bar(labels, revenue_values, color=['blue', 'green'])
plt.ylabel('Price')
plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

In [195]:
predicted_prices =initial_prices


prices_df = pd.DataFrame({
    'Premio ofertado com batente': predicted_prices,
    'Optimized Prices': optimized_prices,
    'lo_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_lote_otim["prob_est"],
    'renewal':renewal
})
prices_df.head(5)

,Predicted Prices,Optimized Prices,lo_bound,up_bound,prob_est,renewal
16458,4275.651790,4275.925369,2294.614,7539.446,0.356873,0.356845
19743,2209.087712,2209.460245,1460.879,4800.031,0.492325,0.492244
1617,2512.541952,2512.763143,2468.403,8110.467,0.307582,0.307546
29835,10427.893052,10428.021098,9983.393,32802.577,0.183314,0.183311
11829,2118.581600,2118.936670,1214.171,3989.419,0.502294,0.502214


In [60]:
prices_df.shape

(100, 2)

In [65]:
df_lote.head(2)

,Act_cost,afinidade,Agravo/Desconto da Otimização,Agrup_regiao_SUCURSAL,Agrup_regiao_TERRITORIAL,amostra,ano_fab,ano_modelo,canal,Carprice,cep,chave,fx_qtd_12m,fx_ir_6m_202302_202307,fx_ir_6m_202308_202401,fx_var_prem_atuarial,fx_var_prem_mkp,fx_rd_atu,class_regiao,classe_bonus,Classe_bonus_TI,classif_corr_comer,cod_amostra,cod_corr,cod_end,Coef_otimização,Coef_otimização_MKP,Comissao,Comissao_IC100,Dentro_Limite,desc_assist,desc_config,dt_ter,Empresa,FABRICA,Faixa_Score_Neuro,Flag_Otimizacao,flag_rns,Flag_sinistro,Grupo_veiculo_new,idade,idade_ult_RNS,Incremento_Estratégia,Incremento_Estratégia_com_batente,ind_cas_tot,ind_rcf_tot,is_casco_ofer,is_dmoral,is_rcdc,is_rcdm,MD_alienado,MD_capac,MD_categoria,MD_comb,MD_desc_grupo,MD_est_civil,MD_fator_ajuste,MD_franquia,MD_idade,MD_idade_veic,MD_ISDM,MD_menor26,MD_modalidade,MD_peso_faixa,MD_peso_pot,MD_Porta,MD_pot_faixa,MD_qtd_assis_cpf,MD_renov_proprio_corr,MD_score_serasa,MD_sexo,MD_sin_5a_cas,MD_sin_5a_rcdm,MD_Subzona,MD_tempo_casa_2,MD_tipo_pessoa,MD_tipo_renov,MD_tipo_renov_ant,MD_uso_comer,MD_uso_veic,md_zona,MONTH,Máximo,Mínimo,Nend,cod_end.1,per_com,per_com_percent,comissao_renov_relatorio,Pr_anterior_TI,Premio_Ofertado_Atuarial,Premio_Ofertado_MKP,Premio_Renovado,Flag_Ofertado,Flag_Renovado,Qtd_Ofertadas,Qtd_Renovadas,perf_marca_perfil,PERIODO,peso,Pr_Garantida_TI,premio_creserva,premio_demais_cob,Premio_IC_100,Premio_IC_100_fla,Premio_IC_100_com_bat,premio_oferecido,Premio_oferecido_com_batente,premio_vidros,Premium Parameter,produto,qtd_assis_cpf,qtd_cas_tot,qtd_rcf_tot,qtd_RNS_20anos,Renewal Demand,Renewal Demand MKP,Risco_as_mod,Risco_casco_mod,Risco_Cobertura,Risco_creserva,risco_dc_mod,Risco_demais_cob,Risco_dm_mod,risco_dmo_mod,Risco_vidros,RNS_Casco_calc,RNS_rcf_calc,RNS_TOTAL,sub_produto,TARIFA,Teve_RNS,TIPO_RENOV_TI,UF,Variação do Bônus,Variação do prêmio do ano anterior pós Otimização,vigencia,YEAR,Premio_Ofertado_Atuarial_prob,Act_cost_Atuarial_prob,comis_ic100,comis_atuarial,comis_atuarial_prob,DA,DA_Valor,Premio_Ofertado_MKP_prob,Act_cost_MKP_prob,comis_MKP,comis_MKP_prob,Apolice_renovada,Premio_Ofertado_Final,Premio_Renovado_Atuarial,Premio_Esperado_Renovado_Atuarial,Premio_Renovado_Earnix,Premio_renov_sem_var_risco,Premio_Renovado_MKP,Premio_Renovado_Final,Premio_Renovado_IC100,Premio_IC100_Earnix,Premio_IC100_MKP,Premio_Renovado_IC100_original,act_cost_renov,act_cost_renov_relat,comis_renov,Sinal_MKP,Flag_Alterado_MKP,Flag_tem_no_relatório,premio_renovado_relatorio,Premio_Renovado_IC100_relatorio,Flag_Renovado_Relatorio
0,1554.464123,NAO,0.05,ITAJAI,PARANA,Teste_+5%,2017,2018,Corretor Mais,57228,88210000,603900501883100001,05 - <=18432,03 - 40-<=060,02 - 20-<=040,"(1,1.1]","(1,1.1]","(0.3,0.4]",BOM,10,10,A,5,1115,0,0.894222,NaN,242.594000,246.154255,1,Assistencia 215 250km,03 - Hatchback,2024-02-02,MAPFRE,PEUGEOT,De 55 a 59.99,Elegivel_MP,RNS > 5anos,SEM SINISTRO,Peugeot 208,36,03 ou+ anos,1.029352,1.000000,0,0,57228.0,100000,1000000,300000,SIM,5,10 - Passeio Nacional,Flex,1414 - 208 HATCH,Casado,100,12 - Reduzida,36,7,12 - 300.000,NAO,Valor de Mercado,Faixa 06 - 1050-1099,08- 7 a 07.99,4,Faixa 07 - 140-159,0,SIM,0901 - 0950,Masculino,0,0,21700-VALE DO ITAJAI-BALNEARIO CAMBORIU,11,PF,2 - Renov MAPFRE sem sinistro,2 - Renov MAPFRE sem sinistro,NAO,Particular,21700-VALE DO ITAJAI-BALNEARIO CAMBORIU,2,2425.950000,1212.960000,0,0,10,0.10,0.01,1212.97,2547.237000,2547.237000,1666.5,Sim,Sim,1,1.0,SIM,RENOVAÇÃO,1050,2848.55,0.000000,349.040000,2461.542555,2673.197117,2425.940000,2848.55,2425.940000,155.400000,2547.237000,215,0,0,0,2,0.361846,0.361846,59.333697,886.447072,Compreensiva,0.000000,98.781492,128.076738,291.105528,33.697121,57.022476,0,0,RNS=0,Automais on-line Puro,2547.237000,NAO,2,SC,0,1.10000,Anual,2024,921.707598,562.476673,246.154255,254.723700,92.170760,0.2685,683.933134,921.707598,562.476673,254.723700,92.170760,6.039005e+12,2513.49,2547.237,2513.490000,921.707598,NaN,2547.237,1666.5,3839.765313,890.699403,890.699403,2461.542555,1554.464123,260